In [1]:
from qiskit.aqua.algorithms import VQE, ExactEigensolver
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from qiskit.chemistry.aqua_extensions.components.variational_forms import UCCSD
from qiskit.aqua.components.variational_forms import RYRZ
from qiskit.chemistry.aqua_extensions.components.initial_states import HartreeFock
from qiskit.aqua.components.optimizers import COBYLA, SPSA, SLSQP
from qiskit import IBMQ, BasicAer, Aer
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.chemistry import FermionicOperator
from qiskit import IBMQ
from qiskit.providers.aer import noise
from qiskit.aqua import QuantumInstance
from qiskit.ignis.mitigation.measurement import CompleteMeasFitter
import numpy as np

In [22]:
h1e = np.load('h1e.npy')
h2e = np.load('h2e.npy')

In [23]:
qubitOp = FermionicOperator(h1=h1e, h2=h2e).mapping(map_type='parity')
qubitOp = qubitOp.two_qubit_reduced_operator(2)

/home/noah/.local/lib/python3.7/site-packages/qiskit/aqua/operators/base_operator.py:182: DeprecationWarning: two_qubit_reduced_operator method is deprecated and it will be removed after 0.6. Now it is moved to the `Z2Symmetries` class as a classmethod. Z2Symmeteries.two_qubit_reduction(num_particles)
  DeprecationWarning)


In [24]:
provider = IBMQ.load_account()
backend = Aer.get_backend("qasm_simulator")
device = provider.get_backend("ibmqx2")
coupling_map = device.configuration().coupling_map
noise_model = noise.device.basic_device_noise_model(device.properties())
quantum_instance = QuantumInstance(backend=backend, shots=2**16, 
                                   noise_model=noise_model, 
                                   coupling_map=coupling_map,
                                   measurement_error_mitigation_cls=CompleteMeasFitter,
                                   cals_matrix_refresh_period=30,)

In [25]:
exact_solution = ExactEigensolver(qubitOp).run()
print("Exact Result:", exact_solution['energy'])
optimizer = SPSA(max_trials=100)
var_form = RYRZ(qubitOp.num_qubits, depth=1, entanglement="linear")
vqe = VQE(qubitOp, var_form, optimizer=optimizer, operator_mode="grouped_paulis")
ret = vqe.run(quantum_instance)
print("VQE Result:", ret['energy'])

Exact Result: -1.6457513110645916


/home/noah/.local/lib/python3.7/site-packages/qiskit/aqua/algorithms/adaptive/vqe/vqe.py:105: DeprecationWarning: operator_mode option is deprecated and it will be removed after 0.6. Now the operator has its own mode, no need extra info to tell the VQE.
  "Now the operator has its own mode, no need extra info to tell the VQE.", DeprecationWarning)


VQE Result: -1.534832966558451
